Please install the necessary package and corpus first. We will need it to initialize an NLP object that will be plugged into SkillNer.

$ pip install skillNer
$ python -m spacy download en_core_web_sm

In [1]:
!pip install skillNer

  Created wheel for skillNer: filename=skillNer-1.0.3-py3-none-any.whl size=25649 sha256=904c44561f6cfb1874d80afe531188e060e9113e9e0509ea8a359b7cebd32ca3
  Stored in directory: c:\users\12288\appdata\local\pip\cache\wheels\eb\a1\1a\735667a07483c24897ef0857e0dcbc9bd9d2c0cd0c840c7d7d
Successfully built skillNer


# Overview of this project
According to my understanding of the assignment, I want to use the mature and efficient NLP model based on the spacy and NER model that is pre-trained model and we needn't pay attention on the label our model to do the training process.

We start first by importing modules, particularly spacy and SkillExtractor. Note that if you are using SkillNer for the first time, it might take a while to download SKILL_DB.

SKILL_DB is SkillNer default skills database. It was built upon EMSI skills database.

In [36]:
# imports
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# read raw skills dataset
rsd = pd.read_csv('Raw_Skills_Dataset.csv')

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


It is worth mentioning that before extracting the final result, we can do some easy data cleaning that would reduce the processing time efficiently. So I try to make the whole dataset into lowercase, and then drop the duplicate skillsets. Using a list to contain the rest and unique skills, 'res' is my result. 

And the SkillNER concludes the cleaning process that are redundant phrases cleaner and punctuation cleaner. Furthermore, using Part of Speech works cleaner and tagging each words. And this process includes that N-gram, 2-gram, and 1-gram matcher to get the best result on the skill set. Finally, we can get the skills we want from the raw dataset. 

In [59]:
rsd['RAW DATA'] = rsd['RAW DATA'].str.lower()
rsd_clean = rsd.drop_duplicates()
l=[]
for job_description in rsd_clean['RAW DATA']:
    try :
        annotations = skill_extractor.annotate(job_description)
        if len(annotations["results"]['full_matches']):
            for i in annotations["results"]['full_matches']:     
                l.append(i['doc_node_value'])
        if len(annotations["results"]['ngram_scored']):
            for i in annotations["results"]['ngram_scored']:
                l.append(i['doc_node_value'])
    except IndexError:
        continue
            
res = []
for i in l:
    if i not in res:
        res.append(i)
res

['java',
 'object oriented',
 'relational database',
 'sql',
 'jpa2',
 'hibernate',
 'mybatis',
 'git',
 'maven',
 'gradle',
 'continuous integration',
 'continuous delivery',
 'development processes',
 'jenkins',
 'bamboo',
 'analytical',
 'conceptual',
 'data models',
 'naming convention',
 'high throughput',
 'access services',
 'profile data',
 'data model',
 'analytics',
 'integrations',
 'enterprise architectures',
 'e',
 'data warehouse',
 'datum mart',
 'profiling',
 'algorithms',
 'python',
 'scala',
 'programming',
 'mysql',
 'nosql',
 'processing engines',
 'apache spark',
 'apache flink',
 'apache kafka',
 'aws kinesis',
 'gcp',
 'cloud platform',
 'unix',
 'linux',
 'scripting',
 'tcp ip',
 'docker',
 'kubernetes',
 'api gateway',
 'load balancers',
 'map',
 'reliable',
 'multicast',
 'paxos',
 'github',
 'microservices',
 'rest apis',
 'writing',
 'ruby',
 'bazel',
 'websocket',
 'redis',
 'javascript',
 'html',
 'css',
 'web ui',
 'libraries',
 'angular',
 'ember',
 'kno

In [60]:
skills = pd.DataFrame(res, columns =['Technology Skills'])
skills.to_csv('skills.csv')